In [1]:
import pandas as pd
import numpy as np
url = "http://www.sefaz.rs.gov.br/ASP/Download/Sitagro/PPR_ATIVO.zip"

In [2]:
header_list = ['Inscrição', 'Data Abertura', 'Categoria', 'CNAE_1', 'CNAE_2', 'CNAE_3', 'Tipo', 'CPF/CNPJ']
tipos = {'Inscrição': str , 'Categoria': str, 'CNAE_1': str, 'CNAE_2': str, 'CNAE_3': str, 'Tipo': str, 'CPF/CNPJ': str}
df = pd.read_csv(url, names=header_list, dtype=tipos , sep=";")

1020723025
96013693000398

##### 5993431091 Muito bom

In [132]:
temp = df['Inscrição'][df['CPF/CNPJ'] == '5993431091'.zfill(14)].reset_index(drop = True)
nivel = 100

In [133]:
df_pesq = pd.DataFrame()
df_pesq = df[df['Inscrição'].isin(temp)]
num_ie = np.count_nonzero(df_pesq['Inscrição'].unique())
num_cpf = np.count_nonzero(df_pesq['CPF/CNPJ'].unique())
flg_ie = 0
flg_cpf = 0
flg_nivel = 0
while (num_ie != flg_ie) & (num_cpf != flg_cpf):
    if flg_nivel == nivel:
        break
    num_ie = np.count_nonzero(df_pesq['Inscrição'].unique())
    num_cpf = np.count_nonzero(df_pesq['CPF/CNPJ'].unique())
    df_pesq = pd.merge(df_pesq, df[df['CPF/CNPJ'].isin(df_pesq['CPF/CNPJ'].values)], how='outer')
    df_pesq = pd.merge(df_pesq, df[df['Inscrição'].isin(df_pesq['Inscrição'].values)], how='outer')
    flg_ie = np.count_nonzero(df_pesq['Inscrição'].unique())
    flg_cpf = np.count_nonzero(df_pesq['CPF/CNPJ'].unique())
    flg_nivel += 1
    print(flg_nivel)
    print(nivel)


1
100
2
100
3
100


In [134]:
import networkx as nx
import json
from networkx.readwrite import json_graph

In [135]:
G = nx.DiGraph()

In [136]:
for index, row in df_pesq.iterrows():
    G.add_node(row['Inscrição'], name=row['Inscrição'], label="" , type="IE" )
    G.add_node(row['CPF/CNPJ'], name=row['CPF/CNPJ'], label=row['Categoria'] , type=row['Tipo'])
for index, row in df_pesq.iterrows():
    G.add_edge(row['Inscrição'], row['CPF/CNPJ'], type=""  )

In [137]:
import os
import sys
import subprocess
path_output = "folder"

In [138]:
if not os.path.exists(path_output):
    os.mkdir(path_output)
with open('viz/template.html', 'r', encoding='utf-8') as template:
    str_html = template.read().replace('<!--GRAFO-->', json.dumps(json_graph.node_link_data(G)))

path_html = os.path.join(path_output, 'grafo.html')
with open(path_html, 'w', encoding='utf-8') as html:
    html.write(str_html)

subprocess.Popen(['C:/Program Files (x86)/Google/Chrome/Application/chrome.exe', os.path.abspath(path_html)])

In [ ]:
del(G)